In [1]:
import os
import pickle
import numpy as np
import itertools
import random
from fastDamerauLevenshtein import damerauLevenshtein
import csv
import json

In [120]:
import sys
sys.path.append('..')
from utils.edit_distance import damerau_levenshtein_distance

## Load dev/test data and lexicon

In [8]:
def load_dataset(dataset_fpath):
    examples = {}
    with open(dataset_fpath, 'r') as fd:
        reader = csv.reader(fd, delimiter='\t')
        for i, row in enumerate(reader):
            if i == 0: continue
            examples[int(row[0])] = row
    return examples

print('Load validation set... ', end='')
val_examples = load_dataset('../data/mimic_synthetic_trans/val.tsv')
print(f'done! {len(val_examples)} examples')

print('Load test set... ', end='')
test_examples = load_dataset('../data/mimic_synthetic_trans/test.tsv')
print(f'done! {len(test_examples)} examples')

print('Load clinspell set... ', end='')
clinspell_examples = load_dataset('../data/mimic_clinspell_new/test.tsv')
print(f'done! {len(clinspell_examples)} examples')

Load validation set... done! 10000 examples
Load test set... done! 10000 examples
Load clinspell set... done! 873 examples


In [9]:
print('Load lexicon... ', end='')
with open('../data/new_vocab/lexicon.json', 'r') as fd:
    dictionary = set(json.load(fd))
print(f'done! {len(dictionary)} words')

Load lexicon... done! 822919 words


## Load results

In [14]:
def get_config_params(config):
    if config == 1:
        length_penalty = 1.0
        beam_sort_linear_ed = True
        beam_final_score_normalize_ed = True
    elif config == 2:
        length_penalty = 1.0
        beam_sort_linear_ed = False
        beam_final_score_normalize_ed = True
    elif config == 3:
        length_penalty = 0.0
        beam_sort_linear_ed = False
        beam_final_score_normalize_ed = False
    else:
        raise ValueError(f'Wrong config number: {config}')
    return length_penalty, beam_sort_linear_ed, beam_final_score_normalize_ed

def load_results(result_fpath):
    with open(result_fpath, 'rb') as fd:
        results = pickle.load(fd)
    return results

def get_correct_counts(results):
    correct = len([r for r in results if r['correct'] == r['output'][0]])
    return correct

def acc_string(results_fpath):
    results = load_results(results_fpath)
    total = len(results)
    correct = get_correct_counts(results)   
    return f'{correct:4d}/{total:5d}={correct/total:.4f}'

### 1. LM only decoding

In [12]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers2"
test_experiments = [('Validation', 'results_val_beam{}_ed0.0_lp1.0_bsFalse_fsFalse.pkl'),
                    ('Test', 'results_test_beam{}_ed0.0_lp1.0_bsFalse_fsFalse.pkl')]

for exp_name, fname_template in test_experiments:
    print(f'[{exp_name} results]')
    for beam in [1, 3, 10, 30]:
        results_fname = fname_template.format(beam)
        results_fpath = os.path.join(results_dpath, results_fname)
        print(f'Beam {beam:2d}: {acc_string(results_fpath)}')

[Validation results]
Beam  1: 7294/10000=0.7294
Beam  3: 7334/10000=0.7334
Beam 10: 7336/10000=0.7336
Beam 30: 7336/10000=0.7336
[Test results]
Beam  1:  361/ 1263=0.2858
Beam  3:  387/ 1263=0.3064
Beam 10:  387/ 1263=0.3064
Beam 30:  387/ 1263=0.3064


### 2. LM + ED decoding

In [15]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam  3, ED  0.3: 7396/10000=0.7396
Config 1, Beam  3, ED  0.7: 7983/10000=0.7983
Config 1, Beam  3, ED  1.0: 8147/10000=0.8147
Config 1, Beam  3, ED  2.0: 7655/10000=0.7655
Config 1, Beam  3, ED  5.0: 4714/10000=0.4714
Config 1, Beam  3, ED 10.0: 3159/10000=0.3159
Config 1, Beam 10, ED  0.3: 7615/10000=0.7615
Config 1, Beam 10, ED  0.7: 8512/10000=0.8512
Config 1, Beam 10, ED  1.0: 8793/10000=0.8793
Config 1, Beam 10, ED  2.0: 8494/10000=0.8494
Config 1, Beam 10, ED  5.0: 5722/10000=0.5722
Config 1, Beam 10, ED 10.0: 4078/10000=0.4078
Config 1, Beam 30, ED  0.3: 7633/10000=0.7633
Config 1, Beam 30, ED  0.7: 8635/10000=0.8635
Config 1, Beam 30, ED  1.0: 8991/10000=0.8991
Config 1, Beam 30, ED  2.0: 8948/10000=0.8948
Config 1, Beam 30, ED  5.0: 6529/10000=0.6529
Config 1, Beam 30, ED 10.0: 4639/10000=0.4639
Config 2, Beam  3, ED  0.3: 7302/10000=0.7302
Config 2, Beam  3, ED  0.7: 7784/10000=0.7784
Config 2, Beam  3, ED  1.0: 8011/10000=0.8011
Config 2, Bea

In [16]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers2"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 2, Beam 30, ED  2.0: 9533/10000=0.9533
Config 2, Beam 30, ED  5.0: 9736/10000=0.9736
Config 3, Beam 30, ED  5.0: 9738/10000=0.9738
[Test results]
Config 2, Beam 30, ED  2.0:  905/ 1263=0.7165
Config 2, Beam 30, ED  5.0: 1022/ 1263=0.8092
Config 3, Beam 30, ED  5.0:  990/ 1263=0.7838


### 3. LM + ED + dict decoding

In [17]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers3"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam  3, ED  0.3: 8118/10000=0.8118
Config 1, Beam  3, ED  1.0: 8751/10000=0.8751
Config 1, Beam  3, ED  2.0: 8946/10000=0.8946
Config 1, Beam 10, ED  0.3: 8222/10000=0.8222
Config 1, Beam 10, ED  1.0: 9148/10000=0.9148
Config 1, Beam 10, ED  2.0: 9392/10000=0.9392
Config 1, Beam 30, ED  0.3: 8234/10000=0.8234
Config 1, Beam 30, ED  1.0: 9199/10000=0.9199
Config 1, Beam 30, ED  2.0: 9556/10000=0.9556
Config 2, Beam  3, ED  2.0: 8967/10000=0.8967
Config 2, Beam  3, ED  5.0: 9086/10000=0.9086
Config 2, Beam  3, ED 10.0: 8320/10000=0.8320
Config 2, Beam 10, ED  2.0: 9403/10000=0.9403
Config 2, Beam 10, ED  5.0: 9550/10000=0.9550
Config 2, Beam 10, ED 10.0: 9120/10000=0.9120
Config 2, Beam 30, ED  2.0: 9537/10000=0.9537
Config 2, Beam 30, ED  5.0: 9744/10000=0.9744
Config 2, Beam 30, ED 10.0: 9481/10000=0.9481
Config 3, Beam  3, ED  2.0: 8978/10000=0.8978
Config 3, Beam  3, ED  5.0: 9080/10000=0.9080
Config 3, Beam  3, ED 10.0: 8277/10000=0.8277
Config 3, Bea

### 4. LM + ED + dict decoding (new dict)

In [18]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers4"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam  3, ED  0.3: 8119/10000=0.8119
Config 1, Beam  3, ED  1.0: 8756/10000=0.8756
Config 1, Beam  3, ED  2.0: 8931/10000=0.8931
Config 1, Beam 10, ED  0.3: 8222/10000=0.8222
Config 1, Beam 10, ED  1.0: 9163/10000=0.9163
Config 1, Beam 10, ED  2.0: 9393/10000=0.9393
Config 1, Beam 30, ED  0.3: 8233/10000=0.8233
Config 1, Beam 30, ED  1.0: 9210/10000=0.9210
Config 1, Beam 30, ED  2.0: 9553/10000=0.9553
Config 2, Beam  3, ED  2.0: 8966/10000=0.8966
Config 2, Beam  3, ED  5.0: 9083/10000=0.9083
Config 2, Beam  3, ED 10.0: 8293/10000=0.8293
Config 2, Beam 10, ED  2.0: 9402/10000=0.9402
Config 2, Beam 10, ED  5.0: 9550/10000=0.9550
Config 2, Beam 10, ED 10.0: 9090/10000=0.9090
Config 2, Beam 30, ED  2.0: 9536/10000=0.9536
Config 2, Beam 30, ED  5.0: 9742/10000=0.9742
Config 2, Beam 30, ED 10.0: 9457/10000=0.9457
Config 3, Beam  3, ED  2.0: 8978/10000=0.8978
Config 3, Beam  3, ED  5.0: 9079/10000=0.9079
Config 3, Beam  3, ED 10.0: 8246/10000=0.8246
Config 3, Bea

### 5. LM + ED (DL distance) + dict decoding (new dict)

In [19]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers5"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam 30, ED  0.3: 8234/10000=0.8234
Config 1, Beam 30, ED  1.0: 9208/10000=0.9208
Config 1, Beam 30, ED  2.0: 9551/10000=0.9551
Config 1, Beam 30, ED  5.0: 8887/10000=0.8887
Config 2, Beam 30, ED  2.0: 9533/10000=0.9533
Config 2, Beam 30, ED  5.0: 9744/10000=0.9744
Config 2, Beam 30, ED 10.0: 9457/10000=0.9457
Config 3, Beam 30, ED  2.0: 9575/10000=0.9575
Config 3, Beam 30, ED  5.0: 9744/10000=0.9744
Config 3, Beam 30, ED 10.0: 9408/10000=0.9408
[Test results]
Config 1, Beam 30, ED  0.3:  555/ 1263=0.4394
Config 1, Beam 30, ED  1.0:  818/ 1263=0.6477
Config 1, Beam 30, ED  2.0:  967/ 1263=0.7656
Config 1, Beam 30, ED  5.0:  953/ 1263=0.7546
Config 2, Beam 30, ED  2.0:  910/ 1263=0.7205
Config 2, Beam 30, ED  5.0:  701/  873=0.8030
Config 2, Beam 30, ED 10.0: 1063/ 1263=0.8416
Config 3, Beam 30, ED  2.0:  882/ 1263=0.6983
Config 3, Beam 30, ED  5.0: 1028/ 1263=0.8139
Config 3, Beam 30, ED 10.0: 1058/ 1263=0.8377


In [20]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers5"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0, 20.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam 30, ED  0.3: 8234/10000=0.8234
Config 1, Beam 30, ED  1.0: 9208/10000=0.9208
Config 1, Beam 30, ED  2.0: 9551/10000=0.9551
Config 1, Beam 30, ED  5.0: 8887/10000=0.8887
Config 2, Beam 30, ED  2.0: 9533/10000=0.9533
Config 2, Beam 30, ED  5.0: 9744/10000=0.9744
Config 2, Beam 30, ED 10.0: 9457/10000=0.9457
Config 2, Beam 30, ED 20.0: 8050/10000=0.8050
Config 3, Beam 30, ED  2.0: 9575/10000=0.9575
Config 3, Beam 30, ED  5.0: 9744/10000=0.9744
Config 3, Beam 30, ED 10.0: 9408/10000=0.9408
Config 3, Beam 30, ED 20.0: 7820/10000=0.7820
[Test results]
Config 1, Beam 30, ED  0.3:  555/ 1263=0.4394
Config 1, Beam 30, ED  1.0:  818/ 1263=0.6477
Config 1, Beam 30, ED  2.0:  967/ 1263=0.7656
Config 1, Beam 30, ED  5.0:  953/ 1263=0.7546
Config 2, Beam 30, ED  2.0:  910/ 1263=0.7205
Config 2, Beam 30, ED  5.0:  701/  873=0.8030
Config 2, Beam 30, ED 10.0: 1063/ 1263=0.8416
Config 2, Beam 30, ED 20.0:  974/ 1263=0.7712
Config 3, Beam 30, ED  2.0:  882/ 1263=0.698

In [21]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers5_new"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0, 20.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 2, Beam 30, ED  5.0: 9461/10000=0.9461
Config 2, Beam 30, ED 10.0: 9361/10000=0.9361
[Test results]
Config 1, Beam 30, ED  0.3:  264/  873=0.3024
Config 1, Beam 30, ED  1.0:  486/  873=0.5567
Config 1, Beam 30, ED  2.0:  618/  873=0.7079
Config 1, Beam 30, ED  5.0:  649/  873=0.7434
Config 2, Beam 30, ED  2.0:  550/  873=0.6300
Config 2, Beam 30, ED  5.0:  701/  873=0.8030
Config 2, Beam 30, ED 10.0:  723/  873=0.8282
Config 2, Beam 30, ED 20.0:  675/  873=0.7732
Config 3, Beam 30, ED  2.0:  549/  873=0.6289
Config 3, Beam 30, ED  5.0:  699/  873=0.8007
Config 3, Beam 30, ED 10.0:  716/  873=0.8202
Config 3, Beam 30, ED 20.0:  668/  873=0.7652


In [22]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers6"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0, 20.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 2, Beam 30, ED  5.0: 9378/10000=0.9378
Config 2, Beam 30, ED 10.0: 8849/10000=0.8849
[Test results]
Config 2, Beam 30, ED  5.0:  679/  873=0.7778
Config 2, Beam 30, ED 10.0:  686/  873=0.7858


In [68]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers6_more"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0, 20.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam 30, ED  1.0: 9088/10000=0.9088
Config 1, Beam 30, ED  2.0: 9157/10000=0.9157
Config 2, Beam 30, ED  5.0: 9375/10000=0.9375
Config 2, Beam 30, ED 10.0: 8667/10000=0.8667
[Test results]
Config 1, Beam 30, ED  1.0:  513/  873=0.5876
Config 1, Beam 30, ED  2.0:  629/  873=0.7205
Config 2, Beam 30, ED  5.0:  706/  873=0.8087
Config 2, Beam 30, ED 10.0:  698/  873=0.7995


In [69]:
results_dpath = "../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws"
test_experiments = [('Validation', 'results_val_beam{}_ed{}_lp{}_bs{}_fs{}.pkl'),
                    ('Test', 'results_test_beam{}_ed{}_lp{}_bs{}_fs{}.pkl')]

for exp_name, fname_template in test_experiments:   
    print(f'[{exp_name} results]')
    for config_num, beam, ed_weight in itertools.product([1, 2, 3], [3, 10, 30], [0.3, 0.7, 1.0, 2.0, 5.0, 10.0, 20.0]):
        lp, bs, fs = get_config_params(config_num)
        results_fname = fname_template.format(beam, ed_weight, lp, bs, fs)
        results_fpath = os.path.join(results_dpath, results_fname)    
        if not os.path.exists(results_fpath):
            continue
#         print(results_fpath)
        print(f'Config {config_num}, Beam {beam:2d}, ED {ed_weight:4.1f}: {acc_string(results_fpath)}')

[Validation results]
Config 1, Beam 30, ED  1.0: 8720/10000=0.8720
Config 1, Beam 30, ED  2.0: 9274/10000=0.9274
Config 2, Beam 30, ED  5.0: 9639/10000=0.9639
Config 2, Beam 30, ED 10.0: 9501/10000=0.9501
[Test results]
Config 1, Beam 30, ED  1.0:  436/  873=0.4994
Config 1, Beam 30, ED  2.0:  593/  873=0.6793
Config 2, Beam 30, ED  5.0:  694/  873=0.7950
Config 2, Beam 30, ED 10.0:  725/  873=0.8305


In [ ]:
# 

# Confusion Matrix between Methods

In [116]:
results_val_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/results_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
correct_val_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/correct_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
results_test_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/results_test_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
correct_test_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/correct_test_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
# results_val_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/results_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
# correct_val_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/correct_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
# results_test_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/results_test_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
# correct_test_fpath1 = '../../exp/cond_indep_typo/mimic_synthetic_12layers6_more/correct_test_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'

# results_val_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/results_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
# correct_val_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/correct_val_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
# results_test_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/results_test_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
# correct_test_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/correct_test_beam30_ed5.0_lp1.0_bsFalse_fsTrue.pkl'
results_val_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/results_val_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
correct_val_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/correct_val_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
results_test_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/results_test_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
correct_test_fpath2 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws/correct_test_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'

results_val_fpath3 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws_ed2/results_val_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
correct_val_fpath3 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws_ed2/correct_val_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
results_test_fpath3 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws_ed2/results_test_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'
correct_test_fpath3 = '../../exp/cond_indep_typo/mimic_synthetic_12layers7_aws_ed2/correct_test_beam30_ed10.0_lp1.0_bsFalse_fsTrue.pkl'

### LM + ED vs LM + ED + dict

In [ ]:
val_conf_strings = confusion_table_strings(results_fpath_test_lmeddict, results_fpath_val_lmeddict2)
test_conf_strings = confusion_table_strings(results_fpath_test_lmeddict, results_fpath_test_lmeddict2)

print('1: LM + ED')
print('2: LM + ED + dict')

print('[Val]' + ' ' * 28 + '[Test]')
for str1, str2 in zip(val_conf_strings, test_conf_strings):
    print(str1 + '  ' + str2)

In [ ]:
test_conf_strings = confusion_table_strings(results_fpath_test_lmeddict, results_fpath_test_lmeddict2)
print('\n'.join(test_conf_strings))

# Show correct statistics

In [118]:
typo_correct_dict

{'carediolgy': 'cardiology',
 'lugns': 'lungs',
 'ecchinocytes': 'echinocytes',
 'procuedure': 'procedure',
 'avening': 'evening',
 'enteracept': 'etanercept',
 'hepaotology': 'hepatology',
 'precipirtouskly': 'precipitously',
 'trachiotomy': 'tracheotomy',
 'responsvie': 'responsive',
 'intubwted': 'intubed',
 'sunglottic': 'subglottic',
 'laternatively': 'alternatively',
 'pylenonephritis': 'pyelonephritis',
 'increaseds': 'increases',
 'qudrant': 'quadrant',
 'noteed': 'noted',
 'specil': 'special',
 'distince': 'distinct',
 'hypokiinesis': 'hypokinesis',
 'hypercholesteromia': 'hypercholesteremia',
 'supportvie': 'supportive',
 'spenomegaly': 'splenomegaly',
 'antiarrrhythmics': 'antiarrhythmics',
 'reporteldy': 'reportedly',
 'myalgais': 'myalgias',
 'imapenem': 'imipenem',
 'placemedt': 'placement',
 'transducion': 'transduction',
 'supralilar': 'suprahilar',
 'neoptlams': 'neoplasm',
 'umcomplicated': 'uncomplicated',
 'arimdex': 'arimidex',
 'acrss': 'across',
 'acetazolamind':

In [121]:
for typo, correct in typo_correct_dict.items():
    ed1, oper1 = damerau_levenshtein_distance(typo, correct)
    ed2 = int(damerauLevenshtein(typo, correct, similarity=False))
    if ed1 != ed2:
        print(f'{typo} / {correct} = ours {ed1} {oper1}, orig {ed2}')

enteracept / etanercept = ours 4 [1, 1, 2, 0], orig 3


In [124]:
examples = clinspell_examples
resultss = [load_results(results_test_fpath1),
            load_results(results_test_fpath2),
            load_results(results_test_fpath3)]
correct_resultss = [load_results(correct_test_fpath1),
                    load_results(correct_test_fpath2),
                    load_results(correct_test_fpath3)]
model_names = ['LM train',
               'LM+EDF train',
               'LM+ED2 train']

total_cnt = {}
correct_cnt = {}
typo_correct_dict = {}
correct_cnt2 = {}
correct_mark = {}  # A/B/C

for i in range(len(resultss[0])):
    ex_id = resultss[0][i]['example_id']
    example = examples[ex_id]
    typo, left, right, correct = example[-4:]
    outputss = [results[i] for results in resultss]
    correct_outputs = [correct_results[i] for correct_results in correct_resultss]
    
    if typo not in correct_cnt:
        correct_cnt[typo] = [0] * len(model_names)
        correct_cnt2[typo] = [0] * len(model_names)
        correct_mark[typo] = [[] for _ in model_names]
        typo_correct_dict[typo] = correct
        total_cnt[typo] = 0
    
    total_cnt[typo] += 1
    for j, output in enumerate(outputss):
        if output['output'][0][3:-4] == correct:
            correct_cnt[typo][j] += 1
            correct_cnt2[typo][j] += 1
            correct_mark[typo][j].append(textcolor("A", "green"))
        elif output['scores'][0] - 5e-6 < correct_outputs[j]['score']:
            correct_cnt2[typo][j] += 1
            correct_mark[typo][j].append(textcolor("B", "yellow"))
        else:
            correct_mark[typo][j].append(textcolor("C", "red"))

print('(A only)')
for name, results in zip(model_names, resultss):
    total = len(results)
    correct = get_correct_counts(results)   
    print(f'{name:15s}: {correct:4d}/{total:5d}={correct/total:.4f}')

print('(A & B)')
for i, name in enumerate(model_names):
    total = len(results)
    correct2 = sum([v[i] for v in correct_cnt2.values()])
    print(f'{name:15s}: {correct2:4d}/{total:5d}={correct2/total:.4f}')
            
print(' ' * 45 + ' '.join([f'{name:>12}' for name in model_names]))

frequent_typos = list(zip(*sorted(total_cnt.items(), key=lambda x: x[1], reverse=True)))[0]
for typo in frequent_typos:
    correct = typo_correct_dict[typo]
    print_str = f'{typo}/{correct}'
    print_str += ' ' * (40 - len(print_str))
    print_str += f'({total_cnt[typo]:2d})'
    for i in range(len(model_names)):
        print_str += f' {correct_cnt[typo][i]:4d}'
        print_str += f'({correct_cnt2[typo][i]})'
        print_str += f'{"".join(correct_mark[typo][i])}' + ' ' * (5 - total_cnt[typo])
    print(print_str)

(A only)
LM train       :  742/  873=0.8499
LM+EDF train   :  729/  873=0.8351
LM+ED2 train   :  730/  873=0.8362
(A & B)
LM train       :  811/  873=0.9290
LM+EDF train   :  814/  873=0.9324
LM+ED2 train   :  799/  873=0.9152
                                                 LM train LM+EDF train LM+ED2 train
lugns/lungs                             ( 5)    5(5)AAAAA    5(5)AAAAA    5(5)AAAAA
responsvie/responsive                   ( 5)    5(5)AAAAA    5(5)AAAAA    5(5)AAAAA
pylenonephritis/pyelonephritis          ( 5)    5(5)AAAAA    1(5)BBABB    5(5)AAAAA
noteed/noted                            ( 5)    5(5)AAAAA    5(5)AAAAA    5(5)AAAAA
hypercholesteromia/hypercholesteremia   ( 5)    5(5)AAAAA    5(5)AAAAA    5(5)AAAAA
spenomegaly/splenomegaly                ( 5)    0(5)BBBBB    3(5)BBAAA    0(5)BBBBB
reporteldy/reportedly                   ( 5)    5(5)AAAAA    5(5)AAAAA    5(5)AAAAA
umcomplicated/uncomplicated             ( 5)    5(5)AAAAA    2(5)BABAB    5(5)AAAAA
arimdex/arimidex 

# Compare Examples

In [75]:
def textbf(text):
    return '\033[1m' + text + '\033[0m'

def textcolor(text, color):
    code = {'red': '31', 'green': '32', 'yellow': '33', 'blue': '34', 'magenta': '35', 'cyan': '36', 'white': '37'}
    return '\033[' + code[color] + 'm' + text + '\033[0m'

def print_results(example, outputss, model_names=None, correct_outputs=None, show_context=False, context_length=10, print_scores=False):
    typo, left, right, correct = example[-4:]
    print(' [Input]')
    print(f'typo / correct : {typo} / {correct}')
    if show_context:
        left = ' '.join(left.split()[-context_length:])
        right = ' '.join(right.split()[:context_length])
        print(f'Context: \"{left + " " + textcolor(textbf(typo), "red") + " " + right}\"')
    
    print(' [Output]')
    if not print_scores:
        for i, outputs in enumerate(outputss):
            outputs = [o[3:-4] for o in outputs['output']]
            output_string = ' '.join([textbf(o) if o == correct else o for o in outputs])
            rank = outputs.index(correct)+1 if correct in outputs else "X"
            if model_names:
                model_name = model_names[i]
            else:
                model_name = 'Output {i+1}'
            if rank == 1:
                model_color = 'green'
            elif rank == 'X':
                model_color = 'red'
            else:
                model_color = 'yellow'
            print(f'{textcolor(model_name, model_color) + "(" + str(rank) + ")":23s}: {output_string}')
    else:
        ranks = []
        for outputs in outputss:
            outputs = [o[3:-4] for o in outputs['output']]
            rank = outputs.index(correct)+1 if correct in outputs else "X"
            ranks.append(rank)
            
        first_line_texts = []
        for i, rank in enumerate(ranks):
            if model_names:
                model_name = model_names[i]
            else:
                model_name = 'Output {i+1}'
            if rank == 1:
                model_color = 'green'
            elif rank == 'X':
                model_color = 'red'
            else:
                model_color = 'yellow'
            first_line_texts.append(f'{textcolor(model_name, model_color) + "(" + str(rank) + ")":<35}' + "  Score      LM      ED")
        print(' | '.join(first_line_texts))
        
        for i in range(len(outputss[0]['output'])):
            texts = []
            for j in range(len(outputss)):
                o = outputss[j]['output'][i][3:-4]
                s = outputss[j]['scores'][i]
                l = outputss[j]['lm_scores'][i]
                e = s - l
                if o == correct:
                    o = f'{textbf(o):<33}'
                else:
                    o = f'{o:<25}'
                texts.append(o + f' {s:7.4f} {l:7.4f} {e:7.4f}')
            print(' | '.join(texts))
            
        texts = []
        for output in correct_outputs:
            s = output['score']
            l = output['lm_score']
            e = s - l
            texts.append(f'{"(Correct)":<25} {s:7.4f} {l:7.4f} {e:7.4f}')
        print(' | '.join(texts))

In [76]:
examples = clinspell_examples
resultss = [load_results(results_test_fpath1),
            load_results(results_test_fpath2)]
correct_resultss = [load_results(correct_test_fpath1),
                    load_results(correct_test_fpath2)]
model_names = ['LM train',
               'LM+ED train']

def print_typo_results(typo, show_context=True, context_length=10, print_scores=False):
    correct = typo_correct_dict[typo]
    print(f'{typo}/{correct} ({total_cnt[typo]} total)')
    print(f'({typo} is {textcolor("in", "green") if typo in dictionary else textcolor("not in", "red")} dictionary)')
    print(f'({correct} is {textcolor("in", "green") if correct in dictionary else textcolor("not in", "red")} dictionary)')
    for name, cnt in zip(model_names, correct_cnt[typo]):
        print(f'- {name:10s}: {cnt} correct')
    print()

    for i in range(len(resultss[0])):
        ex_id = resultss[0][i]['example_id']
        example = clinspell_examples[ex_id]
        outputss = [results[i] for results in resultss]
        correct_outputs = [results[i] for results in correct_resultss]
        if example[-4] == typo:
            print_results(example, outputss, model_names=model_names,
                          correct_outputs=correct_outputs, show_context=show_context, context_length=context_length, print_scores=print_scores)
            print('')

## Easy

In [91]:
print_typo_results('noteed', print_scores=True)

noteed/noted (5 total)
(noteed is not in dictionary)
(noted is in dictionary)
- LM train  : 5 correct
- LM+ED train: 5 correct

 [Input]
typo / correct : noteed / noted
Context: "aeration improved and nebs given hrs by resp exp wheezing noteed in upper lobes bs diminished at bases pt with non-prod"
 [Output]
LM train(1)                 Score      LM      ED | LM+ED train(1)              Score      LM      ED
noted                     -1.4503 -0.6170 -0.8333 | noted                     -2.3779 -0.7113 -1.6667
noticed                   -2.4828 -1.2328 -1.2500 | noticed                   -3.3824 -0.8824 -2.5000
notified                  -2.6045 -0.9378 -1.6667 | notified                  -4.2762 -0.9428 -3.3333
needed                    -3.1236 -0.9808 -2.1429 | notes                     -4.7722 -1.4389 -3.3333
notes                     -3.2550 -1.5883 -1.6667 | notehead                  -5.0046 -2.7824 -2.2222
note                      -3.4810 -1.4810 -2.0000 | notices                   

## Medium ?

In [92]:
print_typo_results('spenomegaly', context_length=15, print_scores=True)

spenomegaly/splenomegaly (5 total)
(spenomegaly is not in dictionary)
(splenomegaly is in dictionary)
- LM train  : 2 correct
- LM+ED train: 2 correct

 [Input]
typo / correct : spenomegaly / splenomegaly
Context: "rendering processing on independent wsclip distinct procedural service by same physician name initial pre adenopathy spenomegaly underlying medical condition year old man with splenomegaly htn and gib reason for this examination"
 [Output]
LM train(X)                 Score      LM      ED | LM+ED train(X)              Score      LM      ED
specifically              -3.0190 -0.7114 -2.3077 | penological               -6.9972 -1.9972 -5.0000
specimens                 -3.6084 -0.6084 -3.0000 | somewhatly                -7.1429 -1.6883 -5.4545
specificity               -3.7669 -0.8502 -2.9167 | speechlessly              -7.3196 -2.7042 -4.6154
specialty                 -3.8408 -0.8408 -3.0000 | penologist                -7.5985 -2.1440 -5.4545
specificality             -3.8465 -

In [93]:
print_typo_results('popletial', context_length=15, print_scores=True)

popletial/popliteal (5 total)
(popletial is not in dictionary)
(popliteal is in dictionary)
- LM train  : 2 correct
- LM+ED train: 5 correct

 [Input]
typo / correct : popletial / popliteal
Context: "past medical history pmh htn mi reanl cell ca peptic ulcer dz hyperlipdemia popletial iliac aa cardiac stress negative lv normal wall motion small fixed distal anterior wall defect"
 [Output]
LM train(X)                 Score      LM      ED | LM+ED train(1)              Score      LM      ED
positional                -3.1760 -0.9033 -2.2727 | popliteal                 -4.5916 -2.5916 -2.0000
optimal                   -3.3280 -0.8280 -2.5000 | political                 -5.5558 -2.5558 -3.0000
positive                  -3.5918 -0.8140 -2.7778 | politically               -6.0475 -1.8808 -4.1667
post-operative            -3.6349 -0.6349 -3.0000 | poplitic                  -6.1665 -2.8332 -3.3333
post-operatively          -3.7699 -0.8287 -2.9412 | poetical                  -6.3122 -2.9788 -3.3

## Hard

In [94]:
print_typo_results('drainags', context_length=15, print_scores=True)

drainags/drainages (4 total)
(drainags is not in dictionary)
(drainages is in dictionary)
- LM train  : 0 correct
- LM+ED train: 1 correct

 [Input]
typo / correct : drainags / drainages
Context: "the fluid collection drain underlying medical condition year old man with whipple with externalized biliary drainags dacron portal vein graft reason for this examination please assess progression of the fluid collection"
 [Output]
LM train(2)                 Score      LM      ED | LM+ED train(2)              Score      LM      ED
drainage                  -0.8806 -0.3251 -0.5556 | drainage                  -1.2893 -0.1782 -1.1111
drainages                 -1.7583 -1.2583 -0.5000 | drainages                 -1.3155 -0.3155 -1.0000
drains                    -2.1320 -0.7034 -1.4286 | drains                    -2.9809 -0.1237 -2.8571
draining                  -2.4824 -0.8157 -1.6667 | draining                  -3.8340 -0.5007 -3.3333
drain                     -2.9716 -0.4716 -2.5000 | drainable 

In [95]:
print_typo_results('persistenting', context_length=15, print_scores=True)

persistenting/persisting (3 total)
(persistenting is not in dictionary)
(persisting is in dictionary)
- LM train  : 0 correct
- LM+ED train: 0 correct

 [Input]
typo / correct : persistenting / persisting
Context: "given po tylenol that decreased the severity of the pain however the pain was still persistenting on his discharge today the patient presents with continued chest pain he describes it as"
 [Output]
LM train(2)                 Score      LM      ED | LM+ED train(3)              Score      LM      ED
persistent                -1.7117 -0.3481 -1.3636 | persistently              -2.8630 -0.5553 -2.3077
persisting                -2.0340 -0.6703 -1.3636 | persistent                -3.1120 -0.3848 -2.7273
persistently              -2.0507 -0.8969 -1.1538 | persisting                -3.5485 -0.8212 -2.7273
presenting                -2.8880 -1.0698 -1.8182 | persistencies             -3.7951 -1.6522 -2.1429
persistence               -2.8929 -1.2263 -1.6667 | presenting               

In [96]:
print_typo_results('increaseds', context_length=15, print_scores=True)

increaseds/increases (2 total)
(increaseds is not in dictionary)
(increases is in dictionary)
- LM train  : 0 correct
- LM+ED train: 0 correct

 [Input]
typo / correct : increaseds / increases
Context: "creatinine elevated from baeline patient was treated with gentle hydration and creatinine returned to baseline increaseds lfts patient was found to have elevated lft on admission patient has known cll so"
 [Output]
LM train(2)                 Score      LM      ED | LM+ED train(2)              Score      LM      ED
increased                 -0.9566 -0.4566 -0.5000 | increased                 -1.6615 -0.6615 -1.0000
increases                 -1.8120 -1.3120 -0.5000 | increases                 -2.3139 -1.3139 -1.0000
increasing                -1.8967 -0.5331 -1.3636 | increasers                -3.1149 -2.2058 -0.9091
increase                  -2.0074 -0.8962 -1.1111 | increasing                -3.3714 -0.6442 -2.7273
increaser                 -2.6005 -1.6005 -1.0000 | increasedly         

In [97]:
print_typo_results('distince', context_length=15, print_scores=True)

distince/distinct (2 total)
(distince is not in dictionary)
(distinct is in dictionary)
- LM train  : 0 correct
- LM+ED train: 0 correct

 [Input]
typo / correct : distince / distinct
Context: "filling defect was noted which was persistent on all images this has rounded apperance with distince superior meniscus and interdigitation of contrast the patient reports history of egd during the last"
 [Output]
LM train(2)                 Score      LM      ED | LM+ED train(2)              Score      LM      ED
distance                  -1.6877 -1.1321 -0.5556 | distance                  -2.5362 -1.4251 -1.1111
distinct                  -1.8493 -1.2937 -0.5556 | distinct                  -2.8616 -1.7505 -1.1111
distances                 -2.2493 -1.2493 -1.0000 | distances                 -3.2128 -1.2128 -2.0000
distinctive               -2.5567 -1.3067 -1.2500 | distanced                 -3.3185 -1.3185 -2.0000
discontinue               -2.5939 -0.9273 -1.6667 | distinctive               -4.07